In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort'])  # Features
y = dataset['Effort']  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Standardize the features to have zero mean and unit variance.
# Standardize the features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
    'neurons_input': (10, 50),
    'neurons_hidden': (10, 50),
    'num_hidden_layers': (1, 5),
    'learning_rate': (1e-5, 1e-2),
    'batch_size': (8, 32),
    'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)

    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]


for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y.iloc[train_index], y.iloc[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )
    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.
    
    best_params = optimizer.max['params']
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)


    # You can now use the trained model for prediction or further evaluation
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
    # Convert y_pred and y_test to NumPy arrays for MMRE calculation
    y_pred = np.array(y_pred).flatten()
    y_test = np.array(y_test)
    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)
    
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 160ms/step
| 1         | -1.987e+0 | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 42ms/step
| 2         | -1.07e+04 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 39ms/step
| 3         | -2.006e+0 | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 35ms/step
| 4         | -2.005e+0 | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 0s 34ms/step
| 5         | -2.005e+0 | 18.95     | 16.78     | 0.002005  | 30.57     | 33.7      | 1.186     |
1/1 [==============================] - 0s 42ms/step
|

1/1 [==============================] - 0s 41ms/step
| 32        | -1.099e+0 | 9.227     | 15.19     | 0.008631  | 29.62     | 25.4      | 4.539     |
1/1 [==============================] - 0s 44ms/step
| 33        | -1.045e+0 | 8.0       | 19.16     | 0.01      | 30.64     | 23.18     | 5.0       |
1/1 [==============================] - 0s 39ms/step
| 34        | -1.133e+0 | 8.265     | 18.43     | 0.007471  | 25.71     | 26.74     | 3.087     |
1/1 [==============================] - 0s 46ms/step
| 35        | -1.147e+0 | 8.0       | 13.16     | 0.01      | 24.71     | 25.93     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 36        | -1.048e+0 | 8.0       | 13.15     | 0.01      | 29.06     | 21.63     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 37        | -1.122e+0 | 8.0       | 11.48     | 0.01      | 34.57     | 23.24     | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 38        | -1.105e+0 | 8.0       | 8.312     

1/1 [==============================] - 0s 46ms/step
| 29        | -4.168e+0 | 14.4      | 17.18     | 0.01      | 10.0      | 37.75     | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 30        | -5.549e+0 | 16.5      | 16.84     | 0.01      | 10.0      | 41.02     | 5.0       |
1/1 [==============================] - 0s 41ms/step
| 31        | -5.639e+0 | 9.348     | 17.37     | 0.009971  | 32.8      | 22.42     | 4.842     |
1/1 [==============================] - 0s 32ms/step
| 32        | -1.36e+04 | 8.0       | 16.62     | 1e-05     | 34.1      | 23.26     | 1.0       |
1/1 [==============================] - 0s 42ms/step
| 33        | -1.333e+0 | 11.89     | 17.73     | 0.00112   | 36.87     | 22.82     | 4.589     |
1/1 [==============================] - 0s 41ms/step
| 34        | -5.893e+0 | 8.0       | 20.0      | 0.01      | 34.77     | 21.59     | 4.795     |
1/1 [==============================] - 0s 45ms/step
| 35        | -4.873e+0 | 13.32     | 18.24     

1/1 [==============================] - 0s 38ms/step
| 26        | -2.534e+0 | 8.0       | 17.51     | 0.007111  | 38.51     | 11.46     | 3.337     |
1/1 [==============================] - 0s 35ms/step
| 27        | -4.289e+0 | 10.69     | 19.75     | 0.002663  | 36.24     | 23.26     | 2.243     |
1/1 [==============================] - 0s 38ms/step
| 28        | -2.401e+0 | 8.017     | 17.51     | 0.005111  | 39.16     | 16.71     | 3.732     |
1/1 [==============================] - 0s 39ms/step
| 29        | -2.695e+0 | 8.0       | 16.59     | 0.009342  | 40.18     | 18.13     | 3.583     |
1/1 [==============================] - 0s 42ms/step
| 30        | -4.757e+0 | 8.0       | 18.45     | 1e-05     | 39.87     | 17.84     | 4.887     |
1/1 [==============================] - 0s 37ms/step
| 31        | -2.562e+0 | 9.249     | 18.9      | 0.008678  | 39.86     | 11.55     | 2.224     |
1/1 [==============================] - 0s 40ms/step
| 32        | -2.722e+0 | 9.019     | 19.18     

1/1 [==============================] - 0s 31ms/step
| 23        | -7.122e+0 | 16.34     | 15.16     | 0.004878  | 13.67     | 10.61     | 1.883     |
1/1 [==============================] - 0s 41ms/step
| 24        | -5.7e+03  | 23.59     | 18.05     | 0.005597  | 23.23     | 48.02     | 3.229     |
1/1 [==============================] - 0s 42ms/step
| 25        | -7.119e+0 | 20.57     | 7.095     | 0.004777  | 31.34     | 14.44     | 2.192     |
1/1 [==============================] - 0s 48ms/step
| 26        | -7.131e+0 | 13.42     | 6.584     | 0.001587  | 31.42     | 41.7      | 4.011     |
1/1 [==============================] - 0s 35ms/step
| 27        | -6.978e+0 | 13.33     | 14.39     | 0.005921  | 10.51     | 43.01     | 2.079     |
1/1 [==============================] - 0s 31ms/step
| 28        | -6.886e+0 | 10.51     | 16.13     | 0.005838  | 46.93     | 17.55     | 1.793     |
1/1 [==============================] - 0s 32ms/step
| 29        | -7.129e+0 | 29.95     | 6.991     

1/1 [==============================] - 0s 46ms/step
| 20        | -7.702e+0 | 8.0       | 20.0      | 0.01      | 26.72     | 50.0      | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 21        | -8.716e+0 | 32.0      | 20.0      | 0.01      | 10.0      | 50.0      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 22        | -8.473e+0 | 18.84     | 20.0      | 0.01      | 18.0      | 50.0      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 23        | -7.653e+0 | 8.0       | 10.6      | 0.01      | 50.0      | 35.08     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 24        | -7.874e+0 | 8.0       | 20.0      | 0.01      | 10.0      | 36.61     | 5.0       |
1/1 [==============================] - 0s 38ms/step
| 25        | -8.24e+03 | 8.193     | 18.82     | 0.008926  | 35.36     | 40.18     | 3.4       |
1/1 [==============================] - 0s 45ms/step
| 26        | -7.581e+0 | 8.0       | 10.3      